In [14]:
from app.services.ntropy import NtropyService
import asyncio
import logging

logger = logging.getLogger(__name__)

# async def process_gocardless_transactions(user_id: str):
#     """
#     Process GoCardless transactions through Ntropy enrichment
    
#     Args:
#         user_id (str): The user ID to process transactions for
#     """
#     try:
#         # Initialize Ntropy service
#         ntropy_service = NtropyService()
        
#         # Start the enrichment process
#         batch_response = await ntropy_service.enrich_transactions(user_id)
        
#         if not batch_response:
#             logger.warning("No transactions to process")
#             return
            
#         batch_id = batch_response.id
        
#         # Poll for results
#         while True:
#             status = await ntropy_service.get_batch_status(batch_id)
            
#             if status["status"] == "complete":
#                 logger.info(f"Batch {batch_id} processing completed")
#                 break
#             elif status["status"] == "error":
#                 logger.error(f"Batch {batch_id} failed: {status.get('error')}")
#                 break
            
#             # Wait before next poll
#             await asyncio.sleep(5)  # Poll every 5 seconds
#             logger.info(f"Progress: {status['progress']}/{status['total']}")
            
#     except Exception as e:
#         logger.error(f"Error processing transactions: {str(e)}", exc_info=True)
#         raise

# await process_gocardless_transactions("user_2tEnUq7rivacYtZnsAXPlC5gi9B")

from supabase import create_client
import os

# Initialize Supabase client
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")
supabase = create_client(supabase_url, supabase_key)

# # Fetch all ntropy transactions
# ntropy_transactions = supabase.table("ntropy_transactions").select("*").execute()

# # Loop through each ntropy transaction
# for ntropy_tx in ntropy_transactions.data:
#     ntropy_id = ntropy_tx['ntropy_id']
    
#     # Extract the required fields from the entities and categories
#     counterparty = ntropy_tx['enriched_data']['entities']['counterparty']['name'] if ntropy_tx['enriched_data']['entities'] and ntropy_tx['enriched_data']['entities']['counterparty'] else None
#     category = ntropy_tx['enriched_data']['categories'] if ntropy_tx['enriched_data']['categories'] else None
    
#     # Find and update the corresponding GoCardless transaction
#     # Note: ntropy_id is stored with underscore in gocardless_transactions
#     gocardless_ntropy_id = ntropy_id
    
#     try:
#         # Update the GoCardless transaction with the new data
#         supabase.table("gocardless_transactions").update({
#             "entity": counterparty,
#             "category": category
#         }).eq("id", gocardless_ntropy_id).execute()
        
#         print(f"Updated transaction with ntropy_id: {gocardless_ntropy_id}")
        
#     except Exception as e:
#         print(f"Error updating transaction {gocardless_ntropy_id}: {str(e)}")



Processing batch 1 (10 transactions)
Updated transaction 15f58c8a-32d2-427c-90d3-2b889714c75cef5598b5-f738-4cb0-bafa-f4a6ff92829f with category: Office Supplies
Updated transaction bfa715da-9366-424e-b829-bb8295662e3eaafe35af-0257-4d55-9c06-e01368c2f2a4 with category: Cost of Sales
Updated transaction 2c37580c-5bd5-45a2-a2bb-2f4371b1199c9ef65f8b-e45f-4417-953c-3041cbf08b1d with category: Sales Revenue
Updated transaction 7bc9c59f-b344-47d8-b63f-df4578a26a2525e0a527-e37a-4412-a70b-a6c99a81d299 with category: Travel Expenses
Updated transaction 36eb7eb1-1c90-4568-aef8-3024dc3416bc9eab1eb7-28eb-4506-9ef7-1905614ec345 with category: Telecommunications Expense
Updated transaction 709394df-5624-41fe-83fd-db132d57d34b3ca1405b-49a4-43d5-ac0e-8f6a35991726 with category: Subscriptions and Memberships
Updated transaction 533a3750-7277-4041-9db9-b2f71bf4ad6d1bb60023-abca-45ce-a4a9-e0aa2c534730 with category: Subscriptions and Licenses
Updated transaction e472515c-519a-41ff-9e4d-969bab6705f32caf079

In [ ]:
from app.services.etl.supabase import get_supabase
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

supabase = await get_supabase()

# Get all transactions
logger.info("Fetching all transactions...")
response = await supabase.table("gocardless_transactions").select("*").execute()
logger.info(f"Found {len(response.data)} transactions to update")

# Update chart_of_accounts to None for each transaction
column_to_clear = "chart_of_accounts"
for transaction in response.data:
    logger.info(f"Updating transaction {transaction['id']}")
    await supabase.table("gocardless_transactions")\
        .update({column_to_clear: None})\
        .eq("id", transaction["id"])\
        .execute()

# Get updated data to verify
logger.info("Fetching updated transactions...")
response = await supabase.table("gocardless_transactions").select("*").execute()
logger.info("Update complete")

# Display the results
response.data

## Simi Search with CoA

In [ ]:

""" embedding invoices table """
from typing import List, Dict, Any
import json

from app.services.etl.vectorise_data import get_embedding, JinaTask, kb_item_to_chunks
from app.services.etl.supabase import get_supabase

supabase = await get_supabase()

response = await supabase.table('invoices').select('*').execute()

for i in response.data:
    print(i['results'])
    results_str = json.dumps(i['results'])
    await kb_item_to_chunks(data_id=i['id'], 
                            data_content=results_str, 
                            user_id='ALL', 
                            title=i['results']['invoice_from']['name'],
                            source_table='invoices',
                            is_tabular=False)

### Import bank data from csv

In [ ]:
import pandasai as pai
import pandas as pd
import ast
from dotenv import load_dotenv
import os 

load_dotenv()

pai_api_key = os.getenv("OPENAI_API_KEY")
if not pai_api_key:
    raise ValueError("PAI_API_KEY is not set")
else:
    print(f"OPENAI_API_KEY is set")

df = pd.read_csv("bank_data_23-25/barclays_072.csv")

# Sample DataFrame
df = pai.DataFrame(df)

pai.api_key.set(pai_api_key)


### extract currency and amount
df['currency'] = df['transactionAmount'].apply(lambda x: ast.literal_eval(x)['currency'])
df['amount'] = df['transactionAmount'].apply(lambda x: float(ast.literal_eval(x)['amount']))
df['amount'] = (df['amount'] * 100).astype(int)
df = df.drop('transactionAmount', axis=1)

## setting datetimeindex
df['bookingDate'] = pd.to_datetime(df['bookingDate'])
df.set_index('bookingDate', inplace=True)
df = df.drop(['valueDate', 'bookingDateTime', 'valueDateTime', 'internalTransactionId'], axis=1)
df = df.rename(columns={'remittanceInformationUnstructured': 'remittanceInfo'})

# df.chat("What are the total expenses for 2024?")


OPENAI_API_KEY is set


### LLM Reconciliation Time

- Add new columns "coa_agent", "coa_reason", "coa_agent_confidence" for LLM


In [ ]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
import time
from typing import Dict, Tuple
import logging
import json


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

load_dotenv()

# Initialize the ChatGroq model
groq_model = ChatGroq(model_name="Llama-3.3-70b-Specdec")
openai_model = ChatOpenAI(model="gpt-4o")


class TransactionClassifier:
    def __init__(self):
        logger.info("Initializing TransactionClassifier")
        self.system_prompt = """
        You are a financial expert responsible for classifying transactions into appropriate chart of accounts.
        For each transaction, you must provide:
        1. The most specific appropriate chart of account
        2. A brief explanation of why this classification was chosen. If confidence score is low, explain why.
        3. A confidence score between 0 and 1 (e.g., 0.95 for high confidence)

        You must respond with valid JSON in the following format only:
        {
            "account": "string",
            "reasoning": "string",
            "confidence": float
        }
        """
        self.last_request_time = 0
        self.rate_limit_delay = 2  # 2 seconds between requests (30 requests/minute)

    def _rate_limit(self):
        """Implement rate limiting"""
        current_time = time.time()
        time_since_last_request = current_time - self.last_request_time
        if time_since_last_request < self.rate_limit_delay:
            delay = self.rate_limit_delay - time_since_last_request
            logger.debug(f"Rate limiting: waiting {delay:.2f} seconds")
            time.sleep(delay)
        self.last_request_time = time.time() 

    def classify_transaction(self, transaction: Dict, chart_of_accounts: list) -> Tuple[str, str, float]:
        """Classify a single transaction using the LLM"""
        logger.info(f"Processing transaction: {transaction}")
        self._rate_limit()

        # Format the transaction details for the LLM
        transaction_prompt = f"""
        Please classify the following transaction:
        Transaction: {transaction}
        
        """

        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=transaction_prompt)
        ]

        try:
            logger.debug("Sending request to LLM")
            # Add more visible print statements
            print("\n=== Sending request to LLM ===")
            response = openai_model.invoke(messages)
            print("\n=== Raw LLM Response ===")
            print(response.content)
            logger.debug(f"Raw LLM response: {response.content}")

            # Try to parse the response as JSON
            try:
                # First, try direct JSON parsing
                result = json.loads(response.content)
                logger.info("Successfully parsed JSON response")
            except json.JSONDecodeError:
                # If direct parsing fails, try to extract JSON from the response
                logger.warning("Direct JSON parsing failed, attempting to extract JSON from response")
                # Look for JSON-like structure in the response
                import re
                json_match = re.search(r'\{.*\}', response.content, re.DOTALL)
                if json_match:
                    result = json.loads(json_match.group())
                    logger.info("Successfully extracted and parsed JSON from response")
                else:
                    raise ValueError("No JSON structure found in response")

            # Validate the response structure
            required_keys = {'account', 'reasoning', 'confidence'}
            if not all(key in result for key in required_keys):
                missing_keys = required_keys - result.keys()
                raise ValueError(f"Missing required keys in response: {missing_keys}")

            logger.info(f"Classification successful: {result['account']}")
            return (
                result['account'],
                result['reasoning'],
                result['confidence']
            )

        except Exception as e:
            logger.error(f"Classification failed: {str(e)}", exc_info=True)
            return (
                "ERROR",
                f"Classification failed: {str(e)}",
                0.0
            )


def process_transactions(df: pd.DataFrame, chart_of_accounts: list) -> pd.DataFrame:
    """Process transactions in DataFrame and add classifications directly"""
    logger.info(f"Starting to process {len(df)} transactions")
    classifier = TransactionClassifier()

    # Create a fresh copy of the DataFrame
    df = df.copy()
    
    # Explicitly clear any previous results
    df['coa_agent'] = None
    df['coa_reason'] = None
    df['coa_confidence'] = None

    for i, (idx, row) in enumerate(df.iterrows()):
        print("row", row)
        logger.info(f"Processing transaction {idx}")
        transaction = row.to_dict()
        transaction['amount'] = transaction['amount']/100
        account, reasoning, confidence = classifier.classify_transaction(transaction, chart_of_accounts)
        
        # Update DataFrame using iloc instead of at
        df.iloc[i, df.columns.get_loc('coa_agent')] = account
        df.iloc[i, df.columns.get_loc('coa_reason')] = reasoning
        df.iloc[i, df.columns.get_loc('coa_confidence')] = confidence
        
        # Print summary of classification
        reason_preview = ' '.join(reasoning.split()[:10]) + '...' if reasoning else 'No reason provided'
        logger.info(f"Transaction {idx}:")
        logger.info(f"Account: {account}")
        logger.info(f"Reason Preview: {reason_preview}")
        logger.info("-" * 50)

    
    # Add verification logging
    processed_count = df[df['coa_agent'].notna()].shape[0]
    logger.info(f"Number of processed transactions: {processed_count}")
    
    logger.info("Finished processing all transactions")
    return df


""" Fetch and store chart of accounts into parsed_accounts """
import json

# Read the JSON file
with open('coa.json', 'r') as file:
    data = json.load(file)

# Extract only the required fields from each account
parsed_accounts = []
for account in data['Accounts']:
    parsed_account = {
        'code': account.get('AccountID', ''),
        'name': account.get('Name', ''),
        'status': account.get('Status', ''),
        'type': account.get('Type', ''),
        'taxtype': account.get('TaxType', ''),
        'description': account.get('Description', ''),  # Note: Not present in sample but included as requested
        'class': account.get('Class', ''),
        'reportingcode': account.get('ReportingCode', '')
    }
    parsed_accounts.append(parsed_account)

transactions = []  # Create empty list to store dictionaries

for index, row in df.iterrows():
    transaction = row.to_dict()
    transactions.append(transaction)

df_processed = process_transactions(df.copy(), parsed_accounts)

df_processed

In [ ]:
"""
feeding per line of transaction
"""

In [14]:
from ntropy_sdk import SDK
import os 
from dotenv import load_dotenv

load_dotenv()

ntropy_api_key = os.getenv("NTROPY_API_KEY")

sdk = SDK(ntropy_api_key)

data = [{
    "id": "4yp49x3tbj9mD8DB4fM8DDY6Yxbx8YP14g565Xketw3tFmn",
    "description": "AMAZON WEB SERVICES",
    "entry_type": "outgoing",
    "amount": 12042.37,
    "currency": "USD",
    "date": "2021-11-01",
    "account_holder_id": "35b927b6-6fda-40aa-93b8-95b47c2b2cad",  # Add this required field
    "location": {
        "country": "US"
    }
}]

r = sdk.batches.create(
    operation="POST /v3/transactions",
    data=data
)


vars(r)

{'id': '288a56df-d4ba-4588-9ac5-613ea961d1d7',
 'operation': 'POST /v3/transactions',
 'status': <BatchStatus.PROCESSING: 'processing'>,
 'created_at': datetime.datetime(2025, 2, 16, 3, 48, 3, 126241, tzinfo=TzInfo(UTC)),
 'updated_at': datetime.datetime(2025, 2, 16, 3, 48, 3, 126241, tzinfo=TzInfo(UTC)),
 'progress': 0,
 'total': 1,
 'request_id': '707ac066de354ea08d95bd4156bc4582'}

In [23]:
vars(vars(sdk.batches.results(id=r.id))['results'][0])

{'entities': Entities(counterparty=Counterparty(id='2c0c799b-d003-30a6-9f53-878f3ebf46aa', name='Amazon Web Services', website='aws.amazon.com', logo='https://logos.ntropy.com/aws.amazon.com', mccs=[], type=<CounterpartyType.ORGANIZATION: 'organization'>), intermediaries=[]),
 'categories': Categories(general='cloud computing', accounting=None),
 'location': Location(raw_address='410 Terry Avenue North, Seattle, Washington, United States', structured=LocationStructured(street='Terry Avenue North', city='Seattle', state='Washington', postcode='98109', country_code='US', country='United States', latitude=47.622318, longitude=-122.336649, google_maps_url='https://www.google.com/maps/search/?api=1&query=47.622318,-122.336649', apple_maps_url='https://maps.apple.com/?q=47.622318,-122.336649', store_number=None, house_number='410')),
 'error': None,
 'created_at': datetime.datetime(2025, 2, 16, 3, 48, 3, 137880, tzinfo=TzInfo(UTC)),
 'id': '4yp49x3tbj9mD8DB4fM8DDY6Yxbx8YP14g565Xketw3tFmn'}

In [26]:
vars(sdk.batches.get(id=r.id))

{'id': '288a56df-d4ba-4588-9ac5-613ea961d1d7',
 'operation': 'POST /v3/transactions',
 'status': <BatchStatus.COMPLETED: 'completed'>,
 'created_at': datetime.datetime(2025, 2, 16, 3, 48, 3, 126241, tzinfo=TzInfo(UTC)),
 'updated_at': datetime.datetime(2025, 2, 16, 3, 48, 3, 373098, tzinfo=TzInfo(UTC)),
 'progress': 1,
 'total': 1,
 'request_id': '8fbf405dfe4c4227b5c0a3906b5a28cc'}

In [ ]:
sdk.batches.create(
    operation="POST /v3/transactions",
    data=transactions
)

In [22]:
vars(r)

{'entities': Entities(counterparty=Counterparty(id='2c0c799b-d003-30a6-9f53-878f3ebf46aa', name='Amazon Web Services', website='aws.amazon.com', logo='https://logos.ntropy.com/aws.amazon.com', mccs=[], type=<CounterpartyType.ORGANIZATION: 'organization'>), intermediaries=[]),
 'categories': Categories(general='cloud computing', accounting=None),
 'location': Location(raw_address='410 Terry Avenue North, Seattle, Washington, United States', structured=LocationStructured(street='Terry Avenue North', city='Seattle', state='Washington', postcode='98109', country_code='US', country='United States', latitude=47.622318, longitude=-122.336649, google_maps_url='https://www.google.com/maps/search/?api=1&query=47.622318,-122.336649', apple_maps_url='https://maps.apple.com/?q=47.622318,-122.336649', store_number=None, house_number='410')),
 'error': None,
 'created_at': datetime.datetime(2025, 2, 16, 1, 27, 24, 536241, tzinfo=TzInfo(UTC)),
 'id': '4yp49x3tbj9mD8DB4fM8DDY6Yxbx8YP14g565Xketw3tFmn',


In [ ]:
" NTROPY "
import requests

ntropy_api_key = os.getenv("NTROPY_API_KEY")
if not ntropy_api_key:
    raise ValueError("NTROPY_API_KEY is not set")
else:
    print(f"NTROPY_API_KEY is set")

""" CREATE NEW ACCOUNT HOLDER """
url = "https://api.ntropy.com/v3/account_holders"
headers = {
    "Accept": "application/json",
    "X-API-KEY": ntropy_api_key,
    "Content-Type": "application/json"
}

data = {
    "id": "35b927b6-6fda-40aa-93b8-95b47c2b2cad",
    "type": "business",
    "name": "Michael Ali",
    "website": "https://flowon.ai",
    "industry": "ai software"
}

response = requests.post(url, headers=headers, json=data)
print(response.json())


NTROPY_API_KEY is set
{'details': 'Account holder with provided ID (35b927b6-6fda-40aa-93b8-95b47c2b2cad) already exists'}


In [116]:
""" NTROPY BATCH PROCESS TRANSACTIONS"""
import uuid

url = "https://api.ntropy.com/v3/batches/"


data = {
        "operation": "POST /v3/transactions",
        "data": transformed_data
    }

response = requests.post(url, headers=headers, json=data)
print(response.json())


{'id': '1a2bc613-111b-49b1-b35c-77e9b1d7a2fc', 'operation': 'POST /v3/transactions', 'status': 'processing', 'created_at': '2025-02-11T18:09:18.598722+00:00', 'updated_at': '2025-02-11T18:09:18.598722+00:00', 'progress': 0, 'total': 37}


In [ ]:
""" NOW CHECK TRANSACTION STATUS """
batch_id = "1a2bc613-111b-49b1-b35c-77e9b1d7a2fc"

url = f"https://api.ntropy.com/v3/batches/{batch_id}/results"


get_batch = requests.get(url, headers=headers)

get_batch.json()

In [122]:
get_batch.json()['results'][0]

{'entities': {'counterparty': {'id': 'e393a472-b8bd-33e6-a889-3221b4eea280',
   'name': 'Driver and Vehicle Licensing Agency',
   'website': 'gov.uk.gov.uk',
   'logo': 'https://logos.ntropy.com/business_icons-government_-_tax_-_tax_payment',
   'type': 'organization'},
  'intermediaries': []},
 'categories': {'general': 'tax payment'},
 'location': {'raw_address': 'Longview Road, Swansea SA6 7JL, United Kingdom',
  'structured': {'street': 'Longview Road',
   'city': 'Swansea',
   'state': 'Swansea',
   'postcode': 'SA6 7JL',
   'country_code': 'GB',
   'country': 'United Kingdom',
   'house_number': None,
   'latitude': 51.62079,
   'longitude': -3.94323,
   'google_maps_url': 'https://www.google.com/maps/search/?api=1&query=51.62079,-3.94323',
   'apple_maps_url': 'https://maps.apple.com/?q=51.62079,-3.94323',
   'store_number': None}},
 'created_at': '2025-02-11T18:09:18.623135+00:00',
 'id': '1177539c-b570-4588-9953-d76ae4647afb'}

In [ ]:

transaction_id = "1177539c-b570-4588-9953-d76ae4647afb"

url = f"https://api.ntropy.com/v3/transactions/{transaction_id}"

get_transaction = requests.get(url, headers=headers)

get_transaction.json()

In [ ]:
import json 

""" DF TO JSON FIELDS """
# Convert DataFrame to JSON
def prepare_df_for_frontend(df):
    # Reset index to make bookingDate a column
    df = df.reset_index()
    
    # Convert datetime to ISO format string
    df['bookingDate'] = df['bookingDate'].dt.strftime('%Y-%m-%dT%H:%M:%S')
    
    # Convert to JSON records format (this gives us a string)
    json_string = df.to_json(orient='records', date_format='iso')
    
    # Parse the JSON string into Python objects (list of dictionaries)
    json_data = json.loads(json_string)
    
    return json_data

json_data = prepare_df_for_frontend(df)



In [ ]:
import uuid 
"""  """
def transform_transaction(transaction, account_holder_id):
    # Transform a single transaction
    return {
        "id": str(uuid.uuid4()),
        "description": transaction["remittanceInfo"],
        "date": transaction["bookingDate"].split("T")[0],
        "amount": abs(transaction["amount"]/100),  # Make amount positive
        "entry_type": "outgoing" if transaction["amount"] < 0 else "incoming",
        "currency": transaction["currency"],
        "account_holder_id": account_holder_id,
        "location": {
            "country": "GB"
        }
    }

# Example usage:
account_holder_id = "35b927b6-6fda-40aa-93b8-95b47c2b2cad"
# Transform all transactions using list comprehension
transformed_data = [
    transform_transaction(transaction, account_holder_id) 
    for transaction in json_data
]

transformed_data

## To evaluate reponses of LLM

In [6]:
df_processed.to_csv("transactions_ai_reconciled.csv")